In [1]:
import numpy as np
import json
import copy

In [91]:
class Job:
    def __init__(self, jid, Sj, rj, dj, wj):
        self.id = jid # job id
        self.S = Sj   # sequence of id of Tasks (list)
        self.r = rj   # release date
        self.d = dj   # due date
        self.w = wj   # weight
    
    def __str__(self):
        return f"id={self.id:>3}  w={self.w:>2}  r:{self.r:>3}  d:{self.d:>3}  " + \
               f"min_duration={self.min_duration():>3}  B:{self.B():>3}  " + \
               f"C:{self.C():>3}  late?{self.C()>self.d:>1}  delay={max(0, self.C()-self.d):>2}  cost={self.cost():>4}"
    
    def min_duration(self):
        return np.sum([tasks[tid].p for tid in self.S])
    
    def min_remaining_duration(self):
        return np.sum([tasks[tid].p for tid in self.S if not tasks[tid].done])
    
    def B(self):
        return tasks[self.S[0]].B # beginning date
    
    def C(self):
        return tasks[self.S[-1]].C # completion date
    
    def cost(self):
        C = self.C()
        T = max(C - self.d, 0)
        U = 1 if T > 0 else 0
        return self.w * (C + alpha*U + beta*T)

class Task:
    def __init__(self, tid, jid, pi, workers):
        self.id = tid          # Task id
        self.jid = jid         # Job id of corresponding Job
        self.p = pi            # processing time
        self.workers = workers # possible Workers for this Task (list)
        self.B = None          # beginning date
        self.C = None          # completion date
        self.running = False
        self.done = False
        self.mid = None        # id of Machine on which task runs
        self.oid = None        # id of Operator doing this task
        
    def __str__(self):
        j = jobs[self.jid]
        l_workers = "["
        for w in self.workers:
            tmp = str(w.mid) + ':' + str(w.oid) + ' '
            l_workers += tmp
        l_workers += ']'
        return f"id={self.id:>3}  job={self.jid:>3}  B:{self.B:>3}  C:{self.C:>3}  " + \
               f"(p:{self.p:>2})  (delay:{self.delay():>2})  " + \
               f"m:{self.mid:>2}  o:{self.oid:>2}  workers:{l_workers}"
    
    def delay(self):
        return int(self.B - (jobs[self.jid].r + np.sum([tasks[tid2].p for tid2 in jobs[self.jid].S if tid2 < self.id])))
        
class Worker:
    def __init__(self, mid, oid):
        self.mid = mid # id of Machine used
        self.oid = oid # id of Operator working
        
    def __str__(self):
        return f"machine={self.mid} operator:{self.oid}"
    
class WorkUnit:
    def __init__(self, time, jid, t_idx, tid, w):
        self.time = time   # time at which WorkUnit is considered to start
        self.jid = jid     # Job id
        self.t_idx = t_idx # index of Task in Job's Tasks sequence
        self.tid = tid     # Task id
        self.w = w         # Worker
        
    def __str__(self):
        return f"{self.time}  job:{self.jid}  task:{self.tid}  worker:{self.w.__str__()}"

In [92]:
def parser_job(job):
    jid = job['job']
    Sj  = job['sequence']
    rj  = job['release_date']
    dj  = job['due_date']
    wj  = job['weight']
    return Job(jid, Sj, rj, dj, wj)

def parser_task(task, jid):
    tid = task['task']
    pi  = task['processing_time']
    workers = []
    for machine in task['machines']:
        mid = machine['machine']
        for operator in machine['operators']:
            oid = operator
            workers.append(Worker(mid, oid))
    return Task(tid, jid, pi, workers)

def parser_inst(inst):
    J = inst['parameters']['size']['nb_jobs']
    I = inst['parameters']['size']['nb_tasks']
    M = inst['parameters']['size']['nb_machines']
    O = inst['parameters']['size']['nb_operators']
    alpha = inst['parameters']['costs']['unit_penalty']
    beta = inst['parameters']['costs']['tardiness']
    
    machines  = {} # storing machines' availability by id
    operators = {} # storing operators' availability by id
    jobs  = (J+1) * [None]                          # list of Jobs
    jobs[0] = Job(jid=0, Sj=[], rj=-1, dj=-1, wj=0) # dummy element 0
    tasks = (I+1) * [None]                          # list of Tasks
    tasks[0] = Task(tid=0, jid=0, pi=0, workers=[]) # dummy element 0
    for job in inst['jobs']:
        jid = job['job']
        jobs[jid] = parser_job(job) # add Job in list of Jobs
        for task in inst['tasks']:
            tid = task['task']
            if tid in jobs[jid].S:
                tasks[tid] = parser_task(task, jid) # add task in list of Tasks
                for machine in task['machines']:
                    mid = machine['machine']
                    for operator in machine['operators']:
                        oid = operator
                        machines[mid]  = True # set machine to available
                        operators[oid] = True # set operator to available
    
    return (J, I, M, O, alpha, beta, jobs, tasks, machines, operators)

In [93]:
def optim():
    time = 0
    while (np.sum([t.done for t in tasks]) < I):
        for t in tasks:
            if t.running:
                if (time - t.B >= t.p):
                    t.running = False # task ends
                    t.done = True     # task is done
                    machines[t.mid]  = True # free machine
                    operators[t.oid] = True # free operator
                    t.C = time              # set completion time
        
        for j in jobs:
            if time >= j.r: # it is past the release date of this Job
                for (t_idx, tid) in enumerate(j.S): # loop over Tasks for this Job
                    t = tasks[tid]      # get Task from its id
                    for w in t.workers: # look for a Worker (machine, operator) to execute Task
                        if operators[w.oid] \
                        and machines[w.mid] \
                        and not t.running \
                        and not t.done \
                        and np.all([tasks[tid2].done for tid2 in j.S[:t_idx]]): # all Job's previous Tasks are done
                            t.running = True # set task to running
                            t.B = time       # set beginning time
                            t.mid = w.mid    # set machine id for task
                            t.oid = w.oid    # set operator id for task
                            machines[w.mid]  = False   # set machine to busy
                            operators[w.oid] = False   # set operator to busy

        time += 1 # time flows

In [113]:
def optim2():
    
    ########## SUBROUTINE ##########
    def get_admissible_work_units():
        work_units = []
        for j in jobs:
            if time >= j.r: # it is past the release date of this Job
                for (t_idx, tid) in enumerate(j.S): # loop over Tasks for this Job
                    t = tasks[tid] # get Task from its id
                    if (not t.running) \
                    and (not t.done) \
                    and np.all([tasks[tid2].done for tid2 in j.S[:t_idx]]): # all Job's previous Tasks are done
                        for w in t.workers: # look for a Worker (machine, operator) to execute Task
                            if operators[w.oid] and machines[w.mid]: # Worker is free
                                work_units.append(WorkUnit(time, j.id, t_idx, tid, w))
        return work_units
    ################################
    
    time = 0
    while (np.sum([t.done for t in tasks]) < I):
        
        ################# UPDATE RUNNING TASKS ##################
        for t in tasks:
            if t.running:
                if (time - t.B >= t.p):
                    t.running = False # task ends
                    t.done = True     # task is done
                    machines[t.mid]  = True # free machine
                    operators[t.oid] = True # free operator
                    t.C = time              # set completion time
        #########################################################
        
        work_units = get_admissible_work_units()
        while len(work_units) > 0: # while we can still start tasks
            work_units = sorted(work_units, key=WU_weight)
            #chosen_work_unit = np.random.choice(work_units)
            chosen_work_unit = work_units[-1] # choose the work unit with the biggest weight to finish it at the earliest
            t = tasks[chosen_work_unit.tid]
            w = chosen_work_unit.w
            t.running = True # set task to running
            t.B = time       # set beginning time
            t.mid = w.mid    # set machine id for task
            t.oid = w.oid    # set operator id for task
            machines[w.mid]  = False   # set machine to busy
            operators[w.oid] = False   # set operator to busy
            
            work_units = get_admissible_work_units()
                    
        time += 1 # time flows

In [114]:
def cost(jobs):
    s = 0
    for job in jobs[1:]:
        C = job.C()
        T = max(C - job.d, 0)
        U = 1 if T > 0 else 0
        s += job.w * (C + alpha*U + beta*T)
    return s

In [115]:
def jsonify(tasks):
    res = []
    for task in tasks[1:]:
        sub_res = {}
        sub_res['task'], sub_res['start'], sub_res['machine'], sub_res['operator'] = task.id, task.B, task.mid, task.oid
        res.append(sub_res)
    return res

In [125]:
def WU_weight(work_unit):
    j = jobs[work_unit.jid]  # corresponding Job
    t = tasks[work_unit.tid] # corresponding Task
    C = work_unit.time + j.min_remaining_duration() # proxy for job completion time
    T = max(C - j.d, 0)
    U = 1 if T > 0 else 0
    return w1*j.w*C + w2*j.w*j.d + w3*j.w*alpha*U + w4*j.w*beta*T

---

In [117]:
in_filename = "instances/tiny.json"
with open(in_filename, 'rb') as f:
    inst = json.load(f)

In [118]:
(J, I, M, O, alpha, beta, jobs, tasks, machines, operators) = parser_inst(inst)
print(f"J={J}\nI={I}\nM={M}\nO={O}\nα={alpha}\nβ={beta}")

J=5
I=25
M=8
O=8
α=6
β=1


In [122]:
(J, I, M, O, alpha, beta, jobs, tasks, machines, operators) = parser_inst(inst)
w1, w2, w3, w4 = 1, -1, 0, 0 # default weights, working OK
optim2()
cost(jobs)

579

In [12]:
best_cost = 10**6
for _ in range(1, 100+1):
    if _ % 10 == 0:
        print(f"{_}... ", end=' ')
        print(best_cost)
    (J, I, M, O, alpha, beta, jobs, tasks, machines, operators) = parser_inst(inst)
    optim2()
    candidate_cost = cost(jobs)
    if candidate_cost <= best_cost:
        best_cost = candidate_cost

10...  465
20...  465
30...  465
40...  465
50...  465
60...  465
70...  465
80...  465
90...  465
100...  465


In [123]:
for j in sorted(jobs[1:], key=lambda j: -j.cost())[:20]:
    print(j)

id=  4  w=14  r:  5  d: 19  min_duration= 14  B:  5  C: 19  late?0  delay= 0  cost= 266
id=  2  w= 9  r:  1  d: 10  min_duration=  9  B:  1  C: 12  late?1  delay= 2  cost= 180
id=  1  w= 6  r:  2  d:  8  min_duration=  6  B:  2  C:  9  late?1  delay= 1  cost=  96
id=  3  w= 3  r:  6  d:  9  min_duration=  3  B:  6  C:  9  late?0  delay= 0  cost=  27
id=  5  w= 1  r:  9  d: 10  min_duration=  1  B:  9  C: 10  late?0  delay= 0  cost=  10


In [124]:
for t in sorted(tasks[1:], key=lambda t: (t.B, t.mid, t.oid))[:30]:
    print(t)

id=  2  job=  2  B:  1  C:  2  (p: 1)  (delay: 0)  m: 6  o: 5  workers:[6:1 6:2 6:3 6:5 ]
id=  1  job=  1  B:  2  C:  3  (p: 1)  (delay: 0)  m: 1  o: 7  workers:[1:1 1:4 1:5 1:7 ]
id=  8  job=  2  B:  2  C:  3  (p: 1)  (delay: 0)  m: 7  o: 6  workers:[4:2 7:2 7:6 ]
id=  9  job=  1  B:  3  C:  4  (p: 1)  (delay: 0)  m: 6  o: 4  workers:[6:1 6:2 6:4 6:8 ]
id= 14  job=  2  B:  3  C:  5  (p: 2)  (delay: 0)  m: 8  o: 8  workers:[1:2 1:6 1:8 5:5 8:2 8:6 8:8 ]
id= 18  job=  2  B:  5  C:  6  (p: 1)  (delay: 0)  m: 5  o: 6  workers:[1:2 1:3 1:5 1:6 5:2 5:3 5:5 5:6 5:7 ]
id=  4  job=  4  B:  5  C:  7  (p: 2)  (delay: 0)  m: 7  o: 7  workers:[2:1 2:2 2:4 2:5 2:7 7:2 7:4 7:5 7:7 7:8 ]
id= 10  job=  1  B:  5  C:  6  (p: 1)  (delay: 1)  m: 8  o: 8  workers:[8:3 8:5 8:8 ]
id= 15  job=  1  B:  6  C:  8  (p: 2)  (delay: 1)  m: 1  o: 8  workers:[1:2 1:4 1:7 1:8 7:4 7:7 7:8 ]
id=  3  job=  3  B:  6  C:  7  (p: 1)  (delay: 0)  m: 4  o: 4  workers:[4:1 4:4 4:7 ]
id=  6  job=  4  B:  7  C:  9  (p: 2)  (dela

---

In [16]:
#out_filename = "solutions/KIRO-huge.json"
#with open(out_filename, 'w') as f:
#    json.dump(jsonify(tasks), f)